<a href="https://colab.research.google.com/github/macho-yoo/KW_AI_Hackathon/blob/main/%ED%85%8C%EC%8A%A4%ED%8A%B8(%ED%85%8C%EC%8A%A4%ED%8A%B83%2BCNN)_ipnyb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/월간 11 운동')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from scipy.stats import skew
from scipy.stats import kurtosis
from sklearn.cluster import KMeans
import math
from tqdm import tqdm
from scipy import fftpack
from numpy.fft import *

train = pd.read_csv('train_features.csv')
train_label = pd.read_csv('train_labels.csv')
train_y = train_label.label

test = pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')

In [ ]:
train_acc, train_gy  = train.iloc[:, 2:5], train.iloc[:, 5:]
test_acc, test_gy = test.iloc[:,2:5], test.iloc[:,5:]

#푸리에 변환 데이터 만들기
train_ft = train.copy()
test_ft = test.copy()

In [ ]:
#푸리에 변환을 사용해보자
def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

def get_fourier(data):
  fft=[]
  for i in tqdm(data['id'].unique()):
    temp=data.loc[data['id']==i]
    for i in data.columns[2:8]:
        temp['ft_'+i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
  return fft

#푸리에 변환 데이터 셋에 적용
train_ft=pd.concat(get_fourier(train))
test_ft=pd.concat(get_fourier(test))

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [ ]:
#푸리에 적용 후 DataFrame 수정
train_ft = train_ft[['acc_x',	'acc_y',	'acc_z', 'ft_acc_x',	'ft_acc_y',	'ft_acc_z', 'gy_x',	'gy_y',	'gy_z', 'ft_gy_x',	'ft_gy_y',	'ft_gy_z']]
test_ft = test_ft[['acc_x',	'acc_y',	'acc_z', 'ft_acc_x',	'ft_acc_y',	'ft_acc_z', 'gy_x',	'gy_y',	'gy_z', 'ft_gy_x',	'ft_gy_y',	'ft_gy_z']]

#푸리에 적용 버전 acc,gy
train_ft_acc, train_ft_gy= train_ft.iloc[:,0:6],train_ft.iloc[:,6:12]
test_ft_acc, test_ft_gy= test_ft.iloc[:,0:6],test_ft.iloc[:,6:12]

In [ ]:
import scipy
!pip install transforms3d
from transforms3d.axangles import axangle2mat

def rolling(data):
    for j in np.random.choice(data.shape[0], int(data.shape[0]*2/3)):
        data[j] = np.roll(data[j], np.random.choice(data.shape[1]), axis= 0)
    return data

def rotation(data):
    axis = np.random.uniform(low=-1, high=1, size=data.shape[1])
    angle = np.random.uniform(low=-np.pi, high=np.pi)
    return np.matmul(data , axangle2mat(axis,angle))

def permutation(data, nPerm=4, mSL=10):
    data_new = np.zeros(data.shape)
    idx = np.random.permutation(nPerm)
    bWhile = True
    while bWhile == True:
        segs = np.zeros(nPerm+1, dtype=int)
        segs[1:-1] = np.sort(np.random.randint(mSL, data.shape[0]-mSL, nPerm-1))
        segs[-1] = data.shape[0]
        if np.min(segs[1:]-segs[0:-1]) > mSL:
            bWhile = False
    pp = 0
    for ii in range(nPerm):
        data_temp = data[segs[idx[ii]]:segs[idx[ii]+1],:]
        data_new[pp:pp+len(data_temp),:] = data_temp
        pp += len(data_temp)
    return(data_new)


def Jitter(data, sigma=0.05):
    myNoise = np.random.normal(loc=0, scale=sigma, size=data.shape)
    return data+myNoise


def combine_aug(data, k, aug_P = 0):
    data_ = data.copy()
    if aug_P == 0:
      """
        if (k+1) % 3 == 0:
            for i in np.random.choice(int(data.shape[0]/600), int(data.shape[0]/600*2/3)):
                data_[600*i:600*(i+1)] = rotation(np.array(data_[600*i:600*(i+1)]))
      """
      if (k+1) % 2 == 0:
            for i in np.random.choice(int(data.shape[0]/600), int(data.shape[0]/600*2/3)):
                data_[600*i:600*(i+1)] = permutation(np.array(data_[600*i:600*(i+1)]))
      if (k+1) % 2 == 1:
            for i in np.random.choice(int(data.shape[0]/600), int(data.shape[0]/600*2/3)):
                data_[600*i:600*(i+1)] = Jitter(np.array(data_[600*i:600*(i+1)]))               
                
    if aug_P != 0:
        pass
    return data_


     |████████████████████████████████| 62 kB 1.0 MB/s 
  Created wheel for transforms3d: filename=transforms3d-0.3.1-py3-none-any.whl size=59373 sha256=130b501683c03b0f331d980488bd435ccb3847decf97f263dbf01c78bd35142a
  Stored in directory: /root/.cache/pip/wheels/b5/b7/93/8985551f83720ce37548a5b543c75380bb707955a9c2c5d28c
Successfully built transforms3d


데이터 변수 증가

In [ ]:
pi = math.pi
dt=0.02 
from math import atan, sqrt

def roll_pitch(data):
    roll = (data.iloc[:,1]/(data.iloc[:,0]**2 + data.iloc[:,2]**2).apply(lambda x : sqrt(x))).apply(lambda x : atan(x))*180/np.pi
    pitch = (data.iloc[:,0]/(data.iloc[:,1]**2 + data.iloc[:,2]**2).apply(lambda x : sqrt(x))).apply(lambda x : atan(x))*180/np.pi
    return pd.concat([roll,pitch], axis=1)

# 원본데이터를 에너지값으로 표현
def get_energy(data):
    energy_ = (data.iloc[:,0]**2+data.iloc[:,1]**2+data.iloc[:,2]**2)**(1/3)
    return energy_ 

def get_energy_acc_gy(data,data_):
  
   gy_acc_Energy =((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)

#미분값을 가져보자
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

def setting(data, data_, case = 0):
    if case == 0:
        for i in range(0, data.shape[0], 600):
            data[i] = data_[i] - data_[i+599]
    else:
        for i in range(0, data.shape[0], 600):
            data[i: i+5] = data_[i: i+5].values - data_[i+594:i+599].values
    return data
        
def get_diff(data, case = 0):
    if case == 0:
        x_dif, y_dif, z_dif = data.iloc[:, 0].diff(), data.iloc[:, 1].diff(), data.iloc[:, 2].diff()
    else:
        x_dif, y_dif, z_dif = data.iloc[:, 0].diff(5), data.iloc[:, 1].diff(5), data.iloc[:, 2].diff(5)
    return pd.concat([setting(x_dif, data.iloc[:, 0], case),
                      setting(y_dif, data.iloc[:, 1], case),
                      setting(z_dif, data.iloc[:, 2], case)], axis= 1)

In [ ]:
def train_dataset(acc_data, gy_data, i, aug_P = 0):
    aug_acc = combine_aug(acc_data, i, aug_P)
    aug_gy = combine_aug(gy_data, i, aug_P)

    energy_acc = get_energy(aug_acc)
    energy_gy  = get_energy(aug_gy)
 
    diff_acc = get_diff(aug_acc)
    diff_gy = get_diff(aug_gy)

    roll_pitch_acc = roll_pitch(aug_acc)
    roll_pitch_gy = roll_pitch(aug_gy)

    #energy_acc_gy = get_energy_between(aug_acc, aug_gy)

    return pd.concat([aug_acc, energy_acc, diff_acc ,roll_pitch_acc, aug_gy, energy_gy, diff_gy, roll_pitch_gy], axis= 1)

def test_dataset(acc_data, gy_data):
    roll_pitch_acc = roll_pitch(acc_data)
    roll_pitch_gy = roll_pitch(gy_data)
    
    diff_acc = get_diff(acc_data)
    diff_gy = get_diff(gy_data)

    energy_acc = get_energy(acc_data)
    energy_gy  = get_energy(gy_data)
    
    #energy_acc_gy = get_energy_between(acc_data, gy_data)
    
    return pd.concat([acc_data, energy_acc, diff_acc, roll_pitch_acc, gy_data, energy_gy, diff_gy, roll_pitch_gy], axis= 1)

스케일러

In [ ]:
import sklearn
from sklearn.preprocessing import StandardScaler

data_for_scaler = test_dataset(train_ft_acc, train_ft_gy) # train data만 사용
scaler = StandardScaler().fit(np.array(data_for_scaler))

data_for_scaler = np.array(data_for_scaler).reshape(-1, 600, data_for_scaler.shape[1])
########################################################################################
test_x = test_dataset(test_ft_acc, test_ft_gy)

test_X = scaler.transform(np.array(test_x)).reshape(-1, 600, test_x.shape[1])

모델링 및 학습

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as L

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from numpy.random import seed

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
def first_model():
    inputs = L.Input(shape = (data_for_scaler.shape[1], data_for_scaler.shape[2]))
    gru1 = L.GRU(256, return_sequences = True, dropout = 0.2)(inputs)
    mp = L.MaxPool1D()(gru1)
    ap = L.AveragePooling1D()(gru1)
    concat1 = L.Concatenate()([mp, ap])
    gru2 = L.GRU(256, return_sequences = True, dropout = 0.2)(concat1)
    GAP = L.GlobalAveragePooling1D()(gru2)
    dense = L.Dense(61, activation = "softmax")(GAP)
    return keras.models.Model(inputs, dense)

In [ ]:
def cnn_model():
    seed(2021)
    tf.random.set_seed(2021)
    
    input_layer =  L.Input(shape = (data_for_scaler.shape[1], data_for_scaler.shape[2]))
    conv1 = keras.layers.Conv1D(filters=128, kernel_size=9, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)
    conv1 = keras.layers.Dropout(rate=0.3)(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=6, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
    conv2 = keras.layers.Dropout(rate=0.4)(conv2)
    
    conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)
    conv3 = keras.layers.Dropout(rate=0.5)(conv3)
    
    gap = keras.layers.GlobalAveragePooling1D()(conv3)
    
    output_layer = keras.layers.Dense(61, activation='softmax')(gap)
    
    return keras.models.Model(inputs=input_layer, outputs=output_layer)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import random

def train_model(model_ = None, epochs = 40, first_rlr = 15, second_rlr = 30, r_seed = 10, aug_P = 0, seed_ = 1):
    # first_rlr : 첫번째로 learning_rate이 감소
    # second_rlr : 두번째로 learning_rate이 감소
    # r_seed : StratifiedKFold seed
    # seed_ : numpy/random seed
    
    result_model = []
    cnt = 0
    train_reshape_acc = np.array(train_ft_acc).reshape(-1, 600, 6)
    train_reshape_gy = np.array(train_ft_gy).reshape(-1, 600, 6)
    
    random.seed(seed_)
    tf.random.set_seed(21)

    spliter = StratifiedKFold(n_splits=10, shuffle = True, random_state = r_seed)
    for train_split_id, valid_split_id in spliter.split(train_reshape_acc, train_y):
        
        train_Y, valid_Y = np.array(pd.get_dummies(train_label.label))[train_split_id], np.array(pd.get_dummies(train_label.label))[valid_split_id]
   
        valid_ACC, valid_GY = train_reshape_acc[valid_split_id].reshape(-1, 6), train_reshape_gy[valid_split_id].reshape(-1, 6)
        valid_x = test_dataset(pd.DataFrame(valid_ACC), pd.DataFrame(valid_GY))
        valid_X = scaler.transform(np.array(valid_x)).reshape(-1, 600, valid_x.shape[1])

        model = model_()
        model.compile(optimizer=keras.optimizers.RMSprop(0.003),
                      loss='categorical_crossentropy', metrics=['accuracy'])
        val_score = 0
        seed_ += 1

        for i in range(epochs):
            
            np.random.seed(seed_*47 + i)
            
            train_ACC, train_GY = train_reshape_acc[train_split_id].reshape(-1, 6), train_reshape_gy[train_split_id].reshape(-1, 6)
            train_x = train_dataset(pd.DataFrame(train_ACC), pd.DataFrame(train_GY), i, aug_P)
            train_X = scaler.transform(np.array(train_x)).reshape(-1, 600, train_x.shape[1])

            train_X_ = train_X.copy()

            train_X_ = rolling(train_X_)

            hist = model.fit(train_X_, train_Y, epochs = 1, validation_data = (valid_X, valid_Y), verbose = 0)

            train_accuracy = hist.history["accuracy"]
            new_val_score = accuracy_score(np.argmax(valid_Y, axis = 1), np.argmax(model.predict(valid_X), axis = 1))
            val_loss = hist.history["val_loss"]

            if i == first_rlr:
                model.compile(optimizer=keras.optimizers.RMSprop(0.003*0.2),
                              loss='categorical_crossentropy', metrics=['accuracy'])

            if i == second_rlr:
                model.compile(optimizer = keras.optimizers.RMSprop(0.003*0.2*0.4),
                             loss='categorical_crossentropy', metrics=['accuracy'])

            print("epoch {} - train_accuracy : {} - validation_loss : {} - validation_accuracy : {}".format(i,
                                                                                                            train_accuracy,
                                                                                                            val_loss,
                                                                                                            new_val_score,
                                                                                                            ))

            if i == 0:
                val_loss_score = val_loss[0]
        
            if val_loss_score >= val_loss[0]:
                val_loss_score = val_loss[0]
                best_model = model
                print("####best_val####")
                    
            if new_val_score >= val_score:
                val_score = new_val_score
                best_model = model
                print("####best_acc####")
        print("####################################################### cycle {} is done".format(cnt))
        result_model.append(best_model)
        cnt+=1
    return result_model


def predict_(model):
    result = []
    for mod in model:
        result.append(mod.predict(test_X))
    predict = np.array(result).mean(axis = 0)
    return predict

def save_model(models, name = '1'):
    cnt = 1
    for model in models:
        model.save(path + "submission/last/weight/" + name + '-{}.h5'.format(cnt))
        cnt +=1


In [ ]:
first_result = train_model(cnn_model, r_seed = 2020, seed_ = 25)

In [ ]:
submission.iloc[:, 1:] = predict_(first_result)
submission

In [ ]:
submission.to_csv('energy2_nonEnergyFT_submission.csv', index=False)